In [1]:
import json
import networkx as nx
import numpy as np

In [2]:
#function to read .json file of the input graph
def readPreferenceGraph(filePath) :
    with open(filePath) as file :
        data = json.load(file)

    G = nx.MultiDiGraph()

    for edge in data["edges"] :
        G.add_edge(edge["source"], edge["target"], opinion=edge["opinion"], questionItem=edge["question item"])
    print(G.number_of_edges())
    return G

In [3]:
def calculateProbability(item) :
    if item == 0 or item == 1 :
        return 1
    elif item == 2 or item == 3 :
        return 0.7
    elif item == 4 or item == 5 :
        return 0
    else :
        return 0.3

In [4]:
def calculateIndexes(G, n) :
    #Direct Sociometric Indexes
    Sp = [0] * n #incoming edges to a node - elections
    Pp = [0] * n #incoming edges to a node - perceptions of elections
    Sn = [0] * n #incoming edges to a node - rejections
    Pn = [0] * n #incoming edges to a node - perceptions of rejections
    Rp = [0] * n #number of cases - reciprocal elections
    Rn = [0] * n #number of cases - reciprocal rejections
    Os = [0] * n #number of cases - feeling opposition (election & rejection)
    Ep = [0] * n #outcoming edges from a node - elections
    En = [0] * n #outcoming edges from a node - rejections
    Pap = [0] * n #number of nodes - have perception of election and get election
    Pan = [0] * n #number of nodes - have perception of rejection and get rejection

    def isReciprocal(node1, node2, weight) :
        for u, v, data in G.edges(node2, data=True) :
            if v == node1 and calculateProbability(data["opinion"]) == weight :
                return True
        return False


    for node in G.nodes() :
        for u,v,data in G.in_edges(node, data=True) :
            if calculateProbability(data["opinion"]) == 1 :
                Sp[node] += 1
                if isReciprocal(u, v, 1) :
                    Rp[node] += 1
                elif isReciprocal(u, v, 0) :
                    Os[node] += 1
            elif calculateProbability(data["opinion"]) == 0.7 :
                Pp[node] += 1
            elif calculateProbability(data["opinion"]) == 0.3 :
                Pn[node] += 1
            else :
                Sn[node] += 1
                if isReciprocal(u, v, 0) :
                    Rn[node] += 1
                elif isReciprocal(u, v, 1) :
                    Os[node] += 1
        
        for u,v,data in G.out_edges(node, data=True) :
            if calculateProbability(data["opinion"]) == 1 :
                Ep[node] += 1
            elif calculateProbability(data["opinion"]) == 0 :
                En[node] += 1
            elif calculateProbability(data["opinion"]) == 0.7 :
                if isReciprocal(u, v, 1) :
                    Pap[node] += 1
            else :
                if isReciprocal(u, v, 0) :
                    Pan[node] += 1
                

    #Compound Sociometric Indexes
    #Individual
    popularity = [item / (n - 1) for item in Sp] 
    antipathy = [item / (n - 1) for item in Sn]
    affectiveConnection = [item1 / item2 for item1, item2 in zip(Rp, Sp)]
    sociometricStatus = [(item1 + item2 - item3 - item4) / (n - 1) for item1, item2, item3, item4 in zip(Sp, Pp, Sn, Pn)]
    positiveExpansion = [item/(n - 1) for item in Ep]
    negativeExpansion = [item/(n - 1) for item in En]
    realisticPerception = [(item1 + item2) / (item3 + item4) for item1, item2, item3, item4 in zip(Pap, Pan, Sp, Sn)]
    
    #Group
    association = sum(Rp) / (n* (n-1))
    dissociation = sum(Rn) / (n * (n-1))
    cohesion =  sum(Rp) / sum(Sp)
    socialIntensity = (sum(Sp) + sum(Sn)) / (n - 1)

    print("\033[1mIndividual Sociometrix Indexes\033[0m")
    print(f"Popularity: {popularity}")
    print(f"Antipathy: {antipathy}")
    print(f"Affective Connection: {affectiveConnection}")
    print(f"Sociometric Status: {sociometricStatus}")
    print(f"Positive Expansion: {positiveExpansion}")
    print(f"Negative Expansion: {negativeExpansion}")
    print(f"Realistic Perception: {realisticPerception}")
    
    print("\n\033[1mGroup Sociometrix Indexes\033[0m")
    print(f"Association: {association}")
    print(f"Dissociation: {dissociation}")
    print(f"Cohesion: {cohesion}")
    print(f"Social Intensity: {socialIntensity}")

In [5]:
def createInteractionGraph(preferenceGraph, threshold, maxInteractions, maxTimestamp) :
    n = preferenceGraph.number_of_nodes()
    activitiesCount = 6

    interactionGraph = []
    interactionGraphIntensity = [0] * (maxTimestamp + 1)
    degrees = [0] * n
   
        
    def analyzeQuiz() :
        def findProbBetween(node1, node2) :
            edges_node1_to_node2 = [calculateProbability(data["questionItem"]) for u, v, data in preferenceGraph.edges(node1, data=True) if v == node2]
            edges_node2_to_node1 = [calculateProbability(data["questionItem"]) for u, v, data in preferenceGraph.edges(node2, data=True) if v == node1]

            if len(edges_node1_to_node2) == 0 and len(edges_node2_to_node1) == 0 :
                return None
            elif len(edges_node1_to_node2) == 0 :
                return np.average(edges_node2_to_node1)
            elif len(edges_node2_to_node1) == 0 :
                return np.average(edges_node1_to_node2)
            else :
                return np.average([np.average(edges_node1_to_node2), np.average(edges_node2_to_node1)])        
        
        #dictionary {edge : probability of existing in the Preference Graph}
        edgesProbabilities = {}
        
        #examine edges between nodes in Preference Graph
        #calculate probability. How strong is the connection between people?
        for node in range(n) :
            for other in range(node + 1, n) :
                prob = findProbBetween(node, other)
                if prob is not None :
                    edgesProbabilities[(node, other)] = prob

        #find possible peers for a student
        #step 1: find connecting links with prob < threshold
        possiblePeers = [[] for _ in range(n)]       
        for edge, prob in edgesProbabilities.items() :
            if prob < threshold :
                possiblePeers[edge[0]].append(edge[1])
                possiblePeers[edge[1]].append(edge[0])

        #step 2: keep all the other nodes as possible peers to interact with
        possiblePeers = list(map(lambda item : list(set(range(n)) - set(item[1] + [item[0]])), enumerate(possiblePeers)))
        
        return possiblePeers
    

    def addEdgesAtTimestamp() :
        numberOfInteractions = np.random.choice([2,3])
        G = nx.Graph()
        G.add_nodes_from(range(n))

        for node in range(n) :
            #find for a node how many additional interactions could we add with respect to maxInteractions parameter
            possibleInteractions = maxInteractions - degrees[node]
            if possibleInteractions <= 0 :
                continue

            possibleInteractions = min(numberOfInteractions, possibleInteractions)

            #choose randomly from possible peers
            peers = np.random.choice(possiblePeers[node], size=possibleInteractions, replace=False)
            
            #split possible ways of interaction in sublists and choose randomly an activity from each sublist
            weights = [np.random.choice(sublist) for sublist in np.array_split(range(activitiesCount), possibleInteractions)]

            #add new interaction to the graph
            for peer, weight in zip(peers, weights) :
                G.add_edge(node, peer, interaction_index=weight)
                    
        interactionGraph.append(G)


    def calculateIntensity(time) :
        for node in range(n) :
            count, sum = (0, 0)
            for u, v, data in interactionGraph[time].edges(node, data=True) :
                count += 1
                sum += data["interaction_index"]
            interactionGraphIntensity[time] += count*sum
        


    # calculateIndexes(G=preferenceGraph, n=n)
    

    possiblePeers = analyzeQuiz()

    #interactions at timetsamp t = 0
    #interactions based on a scale-free network model with m=3
    interactionGraph.append(nx.barabasi_albert_graph(n=preferenceGraph.number_of_nodes(), m=3))
    for u, v in interactionGraph[0].edges() :
        interactionGraph[0][u][v]["interaction_index"] = np.random.choice(range(activitiesCount))
    calculateIntensity(time=0)
    
    for node in range(n) :
        degrees[node] += interactionGraph[0].degree(node) 
    

    #interactions at timestamps t >= 1
    for time in range(1, maxTimestamp+1) :
        addEdgesAtTimestamp()

        for node in range(n) :
            degrees[node] += interactionGraph[time].degree(node) 

        calculateIntensity(time=time)

        if nx.is_empty(interactionGraph[time]) :
            interactionGraph += [nx.Graph() for _ in range(maxTimestamp - time)]
            for i in range(maxTimestamp - time) :
                calculateIntensity(time=i)
            break
    
    print(f"Interaction Graph Intensity: {interactionGraphIntensity}")
    
    return interactionGraph

In [6]:
path = "./preferenceGraph.json"

options = {
    "threshold" : 0.4,
    "maxInteractions" : 24,
    "maxTimestamp" : 5
}

G = createInteractionGraph(preferenceGraph=readPreferenceGraph(path), **options)

8431
Interaction Graph Intensity: [14585, 4338, 4003, 3662, 3481, 2910]


In [7]:
print("Snapshots added...")
for i in range(6) :
    print(f"at t = {i}: {G[i]}")

Snapshots added...
at t = 0: Graph with 100 nodes and 291 edges
at t = 1: Graph with 100 nodes and 196 edges
at t = 2: Graph with 100 nodes and 188 edges
at t = 3: Graph with 100 nodes and 180 edges
at t = 4: Graph with 100 nodes and 170 edges
at t = 5: Graph with 100 nodes and 152 edges


In [8]:
activities = ["to share content, chat in a social network platform",
            "to participate in debate/group presentation & discussion",
            "to play games",
            "to study in group or work on collaborative projects",
            "to be volunteers",
            "to be teammates in a sports team"]

for i,j,k in G[1].edges(data=True) :
    print(f"({i},{j}) => interaction: {activities[k['interaction_index']]}")

(0,32) => interaction: to share content, chat in a social network platform
(0,53) => interaction: to be teammates in a sports team
(1,12) => interaction: to participate in debate/group presentation & discussion
(1,16) => interaction: to be teammates in a sports team
(1,24) => interaction: to play games
(1,46) => interaction: to participate in debate/group presentation & discussion
(1,67) => interaction: to study in group or work on collaborative projects
(2,20) => interaction: to participate in debate/group presentation & discussion
(2,56) => interaction: to be volunteers
(2,71) => interaction: to participate in debate/group presentation & discussion
(3,4) => interaction: to share content, chat in a social network platform
(3,76) => interaction: to study in group or work on collaborative projects
(3,53) => interaction: to study in group or work on collaborative projects
(4,99) => interaction: to participate in debate/group presentation & discussion
(4,97) => interaction: to be voluntee